In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import tensorflow as tf
def process_image(image_path, IMG_SIZE=224):
    """
    Takes image path and returns the image as tensors
    """
    #Read in an image file
    image= tf.io.read_file(image_path)

    #Turn jpg image into numerical Tensor with 3 colour channels (Red, Green, Blue)
    image=tf.io.decode_jpeg(image, channels=3)

    # Convert the colour channel values from 0-255 to 0-1 values
    image= tf.image.convert_image_dtype(image, tf.float32)

    #Resize the image to (224,224)
    image=tf.image.resize(image, size=[IMG_SIZE, IMG_SIZE])
    
    return image

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
images_train = []
labels_train = []

In [4]:
train_fire_path = "/kaggle/input/wildfire-detection-image-data/forest_fire/Training and Validation/fire"
for count, x in enumerate(os.listdir(train_fire_path)):
    images_train.append(process_image(train_fire_path+"/"+x))
    labels_train.append(1)
#     print("Done ", count)

In [5]:
train_nofire_path = "/kaggle/input/wildfire-detection-image-data/forest_fire/Training and Validation/nofire"
for count, x in enumerate(os.listdir(train_nofire_path)):
    images_train.append(process_image(train_nofire_path+"/"+x))
    labels_train.append(0)
#     print("Done ", count, f" {x}")

In [6]:
images_train = np.array(images_train)
labels_train = np.array(labels_train)

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(images_train, labels_train,test_size=0.2)

In [8]:
model = tf.keras.models.load_model("/kaggle/input/trial3model/trial3comp.h5")

In [9]:
from tensorflow.keras.callbacks import EarlyStopping
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=1)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [10]:
model.fit(x = X_train,
         y = y_train,
         batch_size = 32, 
         epochs = 20,
         validation_data = (X_val, y_val),
         callbacks = [early_stopping_callback, tensorboard_callback])

Epoch 1/20
46/46 [==============================] - 21s 146ms/step - loss: 0.0535 - accuracy: 0.9945 - val_loss: 0.2061 - val_accuracy: 0.9373
Epoch 2/20
46/46 [==============================] - 5s 101ms/step - loss: 0.0650 - accuracy: 0.9918 - val_loss: 0.5569 - val_accuracy: 0.8283
Epoch 3/20
46/46 [==============================] - 5s 102ms/step - loss: 0.0616 - accuracy: 0.9918 - val_loss: 0.1263 - val_accuracy: 0.9700
Epoch 4/20
46/46 [==============================] - 5s 100ms/step - loss: 0.0470 - accuracy: 0.9966 - val_loss: 0.1264 - val_accuracy: 0.9728
Epoch 5/20
46/46 [==============================] - 4s 98ms/step - loss: 0.0542 - accuracy: 0.9911 - val_loss: 0.3315 - val_accuracy: 0.9046
Epoch 6/20
46/46 [==============================] - 5s 108ms/step - loss: 0.0583 - accuracy: 0.9918 - val_loss: 0.1070 - val_accuracy: 0.9728
Epoch 7/20
46/46 [==============================] - 5s 100ms/step - loss: 0.0404 - accuracy: 0.9980 - val_loss: 0.0638 - val_accuracy: 0.9891
Epoch 

In [12]:
images_test = []
labels_test = []

In [13]:
test_fire_path = "/kaggle/input/wildfire-detection-image-data/forest_fire/Testing/fire"
for count, x in enumerate(os.listdir(test_fire_path)):
    images_test.append(process_image(test_fire_path+"/"+x))
    labels_test.append(1)
#     print("Done ", count)

In [14]:
test_nofire_path = "/kaggle/input/wildfire-detection-image-data/forest_fire/Testing/nofire"
for count, x in enumerate(os.listdir(test_nofire_path)):
    images_test.append(process_image(test_nofire_path+"/"+x))
    labels_test.append(0)
#     print("Done ", count)

In [15]:
y_preds = model.predict(np.array(images_test))

3/3 [==============================] - 1s 116ms/step


In [16]:
from sklearn.metrics import accuracy_score, f1_score


In [17]:
accuracy_score(y_preds>0.5, np.array(labels_test))

0.9852941176470589

In [18]:
f1_score(y_preds>0.5, np.array(labels_test))

0.9777777777777777